<a href="https://colab.research.google.com/github/Ricardomanuel1/Maestria_Ciencia_de_Datos/blob/main/MACHINE%20LEARNING%20Y%20DEEP%20LEARNING/3_Afinamiento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#los parametros en un MLP son la matriz de pesos y los bayas, si yo defino una capa oculta con 30 neuronas y si la entrada es de dimension 100
#se va necesitar una matriz de 100x30 para conectar la capa de entrada con la capa oculta
#esa matriz es donde ocurre el aprendizaje de una red neuronal MLP, son las matrices las que guardan los pesos
#entre capa y capa esos son los parametros en un red neuronal

In [2]:
#los hiperpametros el número de capas, el número de neuronas,la funcion de activacion,el optimizador,
#tamaños del bacht, el numero de epocas, la tecnica de regularizacion y los valores iniciales de los pesos
#antes de comenzar el entrenamiento
#los hiperparametros pueden hacer que resulten mejores parametros

#**Afinamiento de Hiperparámetros en una Red Neuronal MLP**

- La flexibilidad de las redes neuronales es también uno de sus principales inconvenientes: hay muchos hiperparámetros que modificar.

- Incluso en un MLP básico se puede cambiar el número
de capas, el número de neuronas y el tipo de función de activación a utilizar en cada capa, la lógica de inicialización de pesos, el tipo de optimizador a utilizar, su tasa de aprendizaje, tamaño de batch

Opciones para está búsqueda:

- Convertir el modelo Keras en un estimador Scikit-Learn, y usarGridSearchCV o RandomizedSearchCV para ajustar los
hiperparámetros. Para esto, puede usar las
clases  KerasRegressor y KerasClassifier de SciKeras ( https://github.com/adriangb/scikeras).

- Una mejor opción: usar la biblioteca Keras Tuner, desarrollada para ajustar hiperparámetros en modelos Keras. Esa será la opción mostrada a seguir.

In [1]:
import sys
assert sys.version_info >= (3, 7)

In [2]:
from packaging import version
import sklearn
assert version.parse(sklearn.__version__) >= version.parse("1.0.1")

In [3]:
import tensorflow as tf
assert version.parse(tf.__version__) >= version.parse("2.8.0")

In [4]:
import matplotlib.pyplot as plt

plt.rc('font', size=14)
plt.rc('axes', labelsize=14, titlesize=14)
plt.rc('legend', fontsize=14)
plt.rc('xtick', labelsize=10)
plt.rc('ytick', labelsize=10)

Dataset: MNIST Fashion

In [5]:
import tensorflow as tf
fashion_mnist = tf.keras.datasets.fashion_mnist.load_data()

4422102/4422102 [==============================] - 1s 0us/step


Entrenamiento, validación, prueba

In [6]:
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist

X_train, y_train = X_train_full[:-5000], y_train_full[:-5000]#

X_valid, y_valid = X_train_full[-5000:], y_train_full[-5000:]

In [7]:
X_train.shape

(55000, 28, 28)

In [8]:
X_valid.shape

(5000, 28, 28)

In [9]:
X_test.shape

(10000, 28, 28)

In [10]:
tf.keras.backend.clear_session()
tf.random.set_seed(42)

**Keras Tuner**


- Después de instalar e importar ``keras_tuner``, generalmente como kt, se escribe una función que construye, compila y devuelve un modelo de Keras.
- La función debe tomar un objeto kt.HyperParameters como argumento, lo que permite definir hiperparámetros (enteros, flotantes, strings, etc.) junto con sus posibles rangos de valores,
- Estos hiperparámetros pueden usarse para construir
y compilar el modelo.

La siguiente función construye y compila un MLP para clasificar imágenes de MNIST Fashion, se definen hiperparámetros como el número de capas ocultas (``n_hidden``), el número de neuronas por capa (``n_neurons``), la tasa de aprendizaje y el tipo de optimizador a utilizar :

Instalación Keras Tuner

In [11]:
if "google.colab" in sys.modules:
    %pip install -q -U keras_tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 3.6 MB/s eta 0:00:00


Definición de la función

In [12]:
import keras_tuner as kt

def build_model(hp):
    n_hidden = hp.Int("n_hidden", min_value=0, max_value=8, default=2)
    n_neurons = hp.Int("n_neurons", min_value=16, max_value=256)
    learning_rate = hp.Float("learning_rate", min_value=1e-4, max_value=1e-2,
                             sampling="log")

    optimizer = hp.Choice("optimizer", values=["sgd", "adam"])
    if optimizer == "sgd":
        optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)
    else:
        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Flatten())

    for _ in range(n_hidden):
        model.add(tf.keras.layers.Dense(n_neurons, activation="relu"))

    model.add(tf.keras.layers.Dense(10, activation="softmax"))#es de 10 clases y se sa softmax cuando tienen mas de 2 clases

    model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
                  metrics=["accuracy"])#loss="sparse_categorical_crossentropy" se usa por ser de clasificacion
    return model

- La primera parte de la función define los hiperparámetros. Por ejemplo, ``hp.Int("n_hidden", min_value=0, max_value=8, default=2)`` comprueba si un hiperparámetro llamado ``"n_hidden"``
ya está presente en el objeto HyperParameters hp y, de ser así, devuelve su valor. Si no, registra un nuevo hiperparámetro entero llamado ``"n_hidden"``, cuyos valores posibles van de 0 a 8 (inclusive), y
devuelve el valor predeterminado, que es 2. Cuando ``default`` no es definido, el valor retornado es ``min_value``.
- El hiperparámetro "n_neurons" es registrado de manera similar.
- El hiperparámetro "learning_rate" es
registrado como flotante que oscila entre $10^{-4}$ a $10^{-2}$, y desde y dado ``sampling="log"``
muestreo="log", las tasas de aprendizaje de todas las escalas serán sorteadas por igual.
- El hiperparámetro ``optimizer`` se registra con dos posibles
valores: ``"sgd"`` o ``"adam"`` (el valor predeterminado es sgd).

- La segunda parte de la función construye el modelo usando los
valores de hiperparámetros. Crea un modelo secuencial comenzando con un capa Flatten, seguida del número solicitado de capas ocultas (
determinado por el hiperparámetro ``n_hidden``) usando la activación ReLU y una capa de salida con 10 neuronas (una por clase) usando la
función de activación softmax. Por último, la función compila el modelo y lo devuelve.


Si se desea realizar una búsqueda aleatoria, se puede crear un tuner kt.RandomSearch, pasandole la función build_model al
constructor y llamando al método search() del tuner:

In [13]:
'''objective="val_accuracy" es una metrica que va de cero a 1, uno es perfecto
max_trials=5, numero maximo de experimentos
overwrite=True, sobreescribe los resultados intermedios de las ejecuciones
los resultados seran grabados en la carpeta my_fashion_mnist'''

'objective="val_accuracy" es una metrica que va de cero a 1, uno es perfecto\nmax_trials=5, numero maximo de experimentos\noverwrite=True, sobreescribe los resultados intermedios de las ejecuciones\nlos resultados seran grabados en la carpeta my_fashion_mnist'

In [14]:
#cuando termine el entrenamiento tendremos el mejor modelo

In [15]:
random_search_tuner = kt.RandomSearch(
    build_model, objective="val_accuracy", max_trials=5, overwrite=True,
    directory="my_fashion_mnist", project_name="my_rnd_search", seed=42)#max_trials=5 experiemntos

random_search_tuner.search(X_train, y_train, epochs=10,
                           validation_data=(X_valid, y_valid))

Trial 5 Complete [00h 01m 13s]
val_accuracy: 0.8402000069618225

Best val_accuracy So Far: 0.8695999979972839
Total elapsed time: 00h 07m 34s


- El tuner ``RandomSearch`` llama a build_model() una vez con un
objeto de hiperparámetros vacío, solo para reunir sus especificaciones.
- Luego, en este ejemplo, se ejecuta 5 veces; en cada trial, construye un modelo utilizando hiperparámetros muestreados aleatoriamente dentro de sus respectivos rangos, luego entrena ese modelo durante 10 épocas y lo guarda en un subdirectorio.
- Dado que ``overwrite=True``, el directorio my_rnd_search se elimina antes de iniciar el entrenamiento. Si se ejecuta este código por segunda vez pero con ``overwrite=False`` y ``max_trials=10``, el tuner continuará el afinamiento desde donde se quedó, ejecutando 5 pruebas más
- Por último, dado que el objetivo es "val_accuracy", el tuner prefiere modelos con una accuracy alto en la  validación, por lo que una vez que el tuner termina, se podrá obtener los mejores modelos:

In [ ]:
#no es usual hacer croos validacion con modelos profundos

In [18]:
top3_models = random_search_tuner.get_best_models(num_models=3)#permite encontrar los 3 mejores modelos
best_model = top3_models[0]#muestra el primer modelo

También se puede llamar a get_best_hyperparameters() para obtener
kt.HyperParameters de los mejores modelos:

In [19]:
top3_params = random_search_tuner.get_best_hyperparameters(num_trials=3)#parametros de los 3 modelos
top3_params[0].values  # best hyperparameter values del primer modelo

{'n_hidden': 8,
 'n_neurons': 37,
 'learning_rate': 0.008547485565344062,
 'optimizer': 'sgd'}

- Cada tuner es guiado por un oráculo: antes de cada prueba, el tuner pregunta al oráculo que le indique cuál debería ser la próxima prueba.
- El tuner RandomSearch utiliza RandomSearchOracle, que es bastante básico: simplemente selecciona el siguiente trial al azar.
- Dado que el oráculo realiza un seguimiento de todas
las pruebas, se puede solicitar la mejor y mostrar un resumen:


In [20]:
best_trial = random_search_tuner.oracle.get_best_trials(num_trials=1)[0]
best_trial.summary()

Trial 3 summary
Hyperparameters:
n_hidden: 8
n_neurons: 37
learning_rate: 0.008547485565344062
optimizer: sgd
Score: 0.8695999979972839


También se puede acceder a las métricas directamente:

In [21]:
best_trial.metrics.get_last_value("val_accuracy")

0.8695999979972839

In [ ]:
#ya encontrando los mejores hiperparametros es necesario entrenarlo con el full data

Entrenamiento completo

Si se está satisfecho con el rendimiento del mejor modelo, se puede continuar entrenándolo durante algunas épocas usando el conjunto de entrenamiento completo (X_train_full y
y_train_full), para despúes evaluarlo en el conjunto de prueba y ponerlo en producción.

In [22]:
best_model.fit(X_train_full, y_train_full, epochs=10)
test_loss, test_accuracy = best_model.evaluate(X_test, y_test)

Epoch 1/10
1875/1875 [==============================] - 9s 4ms/step - loss: 0.3613 - accuracy: 0.8690
Epoch 2/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.3528 - accuracy: 0.8710
Epoch 3/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.3444 - accuracy: 0.8739
Epoch 4/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.3393 - accuracy: 0.8768
Epoch 5/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.3373 - accuracy: 0.8765
Epoch 6/10
1875/1875 [==============================] - 9s 5ms/step - loss: 0.3325 - accuracy: 0.8787
Epoch 7/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.3273 - accuracy: 0.8798
Epoch 8/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.3214 - accuracy: 0.8820
Epoch 9/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.3162 - accuracy: 0.8833
Epoch 10/10
313/313 [==============================] - 1s 3ms/step - loss: 0.3832 

# **Elección de parámetros**

**Número de Capas Ocultas**
- Para muchos problemas se puede empezar con sólo una o dos capas ocultas.

- Se puede alcanzar fácilmente una precisión superior al 97 % en el conjunto de datos MNIST utilizando solo una
capa oculta con unos pocos cientos de neuronas y una precisión superior al 98% utilizando
dos capas ocultas con el mismo número de neuronas.

- Para problemas más complejos, se puede aumentar el número de capas ocultas hasta que produzca overfitting en el conjunto de entrenamiento.

- Tareas muy complejas, como clasificación de imágenes o reconocimiento de voz normalmente requieren redes con docenas de capas (o incluso cientos, pero no MLPs), y necesitan una gran cantidad de datos de entrenamiento.

- Transfer learning: rara vez se entrena tales
redes grandes desde cero: es mucho más común reutilizar partes de una
red de última generación previamente entrenada que realiza una tarea similar. La clasificación entonces será mucho más rápida y requerirá muchos menos datos.


**Número de neuronas por capa oculta**

- Las capas de entrada y salida dependen de los datos
- Antes se pensaba que en MLPs debería comenzarse con muchas neuronas e ir disminuyendo gradualmente. La red neuronal para MNIST podía tener 3 capas ocultas, la primera con 300 neuronas, la segunda con 200 y la tercera con 100.
- Esta práctica ha sido abandonada en gran medida porque parece que usar el mismo número de neuronas en todas las capas ocultas funciona igual de bien en la mayoría de casos.
-  En general, dependiendo del conjunto de datos, a veces puede resultar útil hacer la primera capa oculta más grande que las demás.

**Learning Rate, Batch Size, y otros**

*Learning rate*:

- Para encontrar una buena tasa de aprendizaje se entrena
el modelo durante unos cientos de iteraciones, comenzando con un valor muy bajo (por ejemplo, $10^{-5}$)
- Se aumenta gradualmente hasta un valor muy grande (por ejemplo,
10).
- Esto se consigue multiplicando la tasa de aprendizaje por un factor constante en cada iteración (por ejemplo, por $(10/10^{-5})^{1/500}$ para pasar de $10^{-5}$ a 10 en 500
iteraciones).
- Se grafica la pérdida (loss) en función de la tasa de aprendizaje (usando un escala logarítmica), se debería observar que disminuye al principio. Pero después de un tiempo, la tasa de aprendizaje será demasiado grande, por lo que el loss se disparará.
- Copia de seguridad: la tasa de aprendizaje óptima será un poco más baja que el punto en donde el loss comienza a aumentar.
- Se re entrena el modelo con esta tasa de aprendizaje óptima.

*Optimizador*:

Elegir un optimizador mejor que el antiguo descenso de gradiente con mini batches (y ajustar sus hiperparámetros) es bastante importante. Se verán opciones en otros jupyters.

*Batch*:

- El tamaño del batch puede tener un impacto significativo en el rendimiento de su modelo.
- El principal beneficio de utilizar batches grandes es que las GPU pueden procesarlos eficientemente.
- Sin embargo, hay un problema: en la práctica, los batches grandes a menudo conducen a inestabilidades del entrenamiento, especialmente al inicio del mismo, y el modelo resultante puede no generalizar bien
- Según LeCun (2018) se debe usar tamaños de batch de hasta 32.
- Dominic Masters y Carlo Luschi, investigaron que el uso de pequeños
batches (de 2 a 32) era preferible ya que conducían a mejores modelos en menos tiempo de entrenamiento.
-  Para contrariar esto, Elad Hoffer et al.
y Priya Goyal et al. demostraron que era posible utilizar tamaños muy grandes de batch (hasta 8,192) junto con varias técnicas asociadas a la alteración de la tasa de aprendizaje. Obtuvieron tiempos de entrenamiento cortos, sin ninguna brecha de generalización.
- Una estrategia es intentar a utilizar un tamaño de batch grande, con una pequeña tasa de aprendizaje y luego aumentándola, y si el entrenamiento es inestable o el rendimiento final es decepcionante, utilice un batch pequeño.

*Función de activación*

En general, la función de activación ReLU será un buen valor predeterminado para todas las capas ocultas, pero para la capa de salida depende de la tarea a resolver.

*Número de iteraciones*

En la mayoría de los casos, no es necesario que el número de iteraciones sea ajustada, utilice early stopping.

In [ ]:
#batch cuantos ejemplos se va utlizar para actualizar